In [1]:
! git clone https://github.com/IlyaGusev/summarus.git

Cloning into 'summarus'...
remote: Enumerating objects: 1484, done.
remote: Counting objects: 100% (118/118), done.
remote: Compressing objects: 100% (39/39), done.
remote: Total 1484 (delta 99), reused 96 (delta 79), pack-reused 1366
Receiving objects: 100% (1484/1484), 504.99 KiB | 3.16 MiB/s, done.
Resolving deltas: 100% (988/988), done.


In [6]:
! git clone https://github.com/wdariaw/ru_articles_summarization

Cloning into 'ru_articles_summarization'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 10 (delta 1), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (10/10), 23.09 KiB | 482.00 KiB/s, done.


## Подготовка данных

In [1]:
%cd ru_articles_summarization 

/home/ichuviliaeva/is_project/ru_articles_summarization


In [2]:
%ls

evaluate.py  lid.176.ftz.1  README.md         ru_articles_summarization.ipynb
lid.176.ftz  __pycache__/   requirements.txt


In [49]:
import os
import sys
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from datasets import Dataset, DatasetDict
import razdel
import fasttext

from evaluate import print_metrics, postprocess

In [4]:
%ls /DATA/ichuviliaeva/project_data

articles_part_0.csv  articles_part_5.csv  predictions.txt  val_data.json
articles_part_1.csv  articles_part_6.csv  query_list.txt   visited_set.txt
articles_part_2.csv  articles_part_7.csv  targets.txt      vocab.bpe
articles_part_3.csv  articles_part_8.csv  test_data.json
articles_part_4.csv  encoder.json         train_data.json


In [50]:
! wget https://dl.fbaipublicfiles.com/fasttext/supervised-models/lid.176.ftz
lang_detector = fasttext.load_model('lid.176.ftz')

data_path = '/DATA/ichuviliaeva/project_data'
data = []
for file_name in os.listdir(data_path):
    if 'articles' not in file_name:
        continue
    data_part = pd.read_csv(os.path.join(data_path, file_name))

    # Checking that article and abstract are written in russian
    for article, abstract in tqdm(data_part[['article', 'abstract']].to_numpy()):
        if lang_detector.predict(article.replace('\n', ' '), k=1)[0][0] == '__label__en' or \
            lang_detector.predict(abstract.replace('\n', ' '), k=1)[0][0] == '__label__en':
            continue
        data.append([article, abstract])

data = pd.DataFrame(data, columns=['article', 'abstract'])

data.head()

--2023-04-17 09:14:31--  https://dl.fbaipublicfiles.com/fasttext/supervised-models/lid.176.ftz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 108.156.22.45, 108.156.22.14, 108.156.22.118, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|108.156.22.45|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 938013 (916K) [binary/octet-stream]
Saving to: ‘lid.176.ftz’

lid.176.ftz         100%[===================>] 916.03K  --.-KB/s    in 0.1s    

2023-04-17 09:14:32 (6.03 MB/s) - ‘lid.176.ftz’ saved [938013/938013]



  0%|          | 0/800 [00:00<?, ?it/s]

  0%|          | 0/5000 [00:00<?, ?it/s]

  0%|          | 0/5000 [00:00<?, ?it/s]

  0%|          | 0/5000 [00:00<?, ?it/s]

...

  0%|          | 0/5000 [00:00<?, ?it/s]

  0%|          | 0/5000 [00:00<?, ?it/s]

  0%|          | 0/5000 [00:00<?, ?it/s]

  0%|          | 0/5000 [00:00<?, ?it/s]

,article,abstract
0,Состав и калорийность\nПлоды айвы достаточно н...,"Айву, которую часто называют ложным яблоком вс..."
1,В последние годы при госпитализации больных с ...,Исследовалась частота ассоциации заболеваний щ...
2,"Введение. Одной из серьезных проблем, стоящих ...",Цель исследования - провести анализ динамики с...
3,Актуальность проблемы. На современном этапе пр...,В статье проанализированы данные о впервые выя...
4,Сохранение здоровья детей в Российской Федерац...,В статье представлены краткие данные о распрос...


In [51]:
display(data)

,article,abstract
0,Состав и калорийность\nПлоды айвы достаточно н...,"Айву, которую часто называют ложным яблоком вс..."
1,В последние годы при госпитализации больных с ...,Исследовалась частота ассоциации заболеваний щ...
2,"Введение. Одной из серьезных проблем, стоящих ...",Цель исследования - провести анализ динамики с...
3,Актуальность проблемы. На современном этапе пр...,В статье проанализированы данные о впервые выя...
4,Сохранение здоровья детей в Российской Федерац...,В статье представлены краткие данные о распрос...
...,...,...
38670,Одним из значимых симптомов в структуре дефект...,Работа посвящена рассмотрению проблемы коррекц...
38671,Актуальность обращения к методу сенсорной инте...,Представлены результаты экспериментального исс...
38672,"По определению, принятому в научной литературе...",В статье рассматриваются условия успешного обу...
38673,В настоящее время в логопедии понятие «звуко-с...,В статье рассматривается актуальная проблема д...


In [53]:
data.drop_duplicates(inplace=True)

In [54]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(data, test_size=0.2, random_state=42)
test, val = train_test_split(test, test_size=0.5, random_state=42)

In [55]:
display(val)

,article,abstract
36246,"прочность, удар, разрушение, частица, измельче...",Среди технологического оборудования для измель...
33141,Актуальность научного исследования. Проблема в...,Проведено изучение морфологических особенносте...
23860,Современные направления развития высшего образ...,В работе представлены результаты проведенного ...
8364,Вопрос о праве требует понятийно-философского ...,"Мы находимся на переломном этапе истории, когд..."
13153,Введение\nздоровых детей раннего возраста при ...,Целью исследования являлась разработка подходо...
...,...,...
6920,I I роисходящие в мире преобразования в эконом...,"Рассматривается сущность, специфика и структур..."
2592,В современном обществе все более ясно осознает...,В статье приводятся результаты исследования ин...
11425,"Основным видом органического вяжущего, применя...","Проведен анализ проблем, связанных с производс..."
17884,Специфика деятельности таких больших библиотеч...,В статье раскрывается структура информационног...


In [13]:
print(len(train['abstract']))

30940


In [94]:
print(len(train.iloc[8]['article']))

20633


In [110]:
# prepearing for BART
start_prefix = '/DATA/ichuviliaeva/project_data/'
datasets = [train, val, test]
for i, out_prefix in enumerate(['train', 'val', 'test']):
    num_stories = len(datasets[i]['article'])
    
    with open(start_prefix + out_prefix + '.source', 'wt') as source_file, open(start_prefix + out_prefix + '.target', 'wt') as target_file:
        for idx, s in enumerate(datasets[i]['article']):
            if idx % 1000 == 0:
                print("Writing story %i of %i; %.2f percent done" % (idx, num_stories, float(idx)*100.0/float(num_stories)))

            source_file.write(datasets[i].iloc[idx]['article'].replace('\n', ' ') + '\n')
            target_file.write(datasets[i].iloc[idx]['abstract'].replace('\n', ' ') + '\n')

print("Finished writing files")

Writing story 0 of 30476; 0.00 percent done
Writing story 1000 of 30476; 3.28 percent done
Writing story 2000 of 30476; 6.56 percent done
Writing story 3000 of 30476; 9.84 percent done
Writing story 4000 of 30476; 13.13 percent done
Writing story 5000 of 30476; 16.41 percent done
Writing story 6000 of 30476; 19.69 percent done
Writing story 7000 of 30476; 22.97 percent done
Writing story 8000 of 30476; 26.25 percent done
Writing story 9000 of 30476; 29.53 percent done
Writing story 10000 of 30476; 32.81 percent done
Writing story 11000 of 30476; 36.09 percent done
Writing story 12000 of 30476; 39.38 percent done
Writing story 13000 of 30476; 42.66 percent done
Writing story 14000 of 30476; 45.94 percent done
Writing story 15000 of 30476; 49.22 percent done
Writing story 16000 of 30476; 52.50 percent done
Writing story 17000 of 30476; 55.78 percent done
Writing story 18000 of 30476; 59.06 percent done
Writing story 19000 of 30476; 62.34 percent done
Writing story 20000 of 30476; 65.63 p

In [96]:
!git clone https://github.com/facebookresearch/fairseq.git

fatal: destination path 'fairseq' already exists and is not an empty directory.


In [97]:
# to bpe

In [98]:
!ls fairseq/examples/roberta

commonsense_qa			__pycache__
config				README.custom_classification.md
fb_multilingual			README.glue.md
multiprocessing_bpe_encoder.py	README.md
preprocess_GLUE_tasks.sh	README.pretraining.md
preprocess_RACE.py		README.race.md
preprocess_RACE.sh		wsc


In [74]:
%cd fairseq

/home/ichuviliaeva/is_project/fairseq


In [111]:
import fairseq

In [112]:
os.environ['CUDA_VISIBLE_DEVICES'] = '2'

In [19]:
!wget https://www.dropbox.com/s/b2auu9dhrm2wj0p/gazeta_mbart_checkpoint_600_160.tar.gz \
    -O /DATA/ichuviliaeva/project_data/gazeta_mbart_checkpoint_600_160.tar.gz

--2023-04-17 07:05:08--  https://www.dropbox.com/s/b2auu9dhrm2wj0p/gazeta_mbart_checkpoint_600_160.tar.gz
Resolving www.dropbox.com (www.dropbox.com)... 162.125.70.18, 2620:100:6026:18::a27d:4612
Connecting to www.dropbox.com (www.dropbox.com)|162.125.70.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/b2auu9dhrm2wj0p/gazeta_mbart_checkpoint_600_160.tar.gz [following]
--2023-04-17 07:05:09--  https://www.dropbox.com/s/raw/b2auu9dhrm2wj0p/gazeta_mbart_checkpoint_600_160.tar.gz
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc10fcd470b1d08525e3d3154740.dl.dropboxusercontent.com/cd/0/inline/B6Vz0iY2vOlUW9H1V1DJaLSBjgGdSitDsTpWNt_Pfv6Bl6PdCEzL0d-ocQ-IV0UFReZQANnemFfDN9dUvq41uF_mLMZlkyoSa4G1VyXp4p1AdEfqMEQzfft5l0E7NU2Z01rtmVe_lmVOxet5ysfIEGKeUcSwadwLDQgEUgsdBn3V6Q/file# [following]
--2023-04-17 07:05:09--  https://uc10fcd470b1d08525e3d3154740.dl.dropboxusercontent.com/cd/0/inline

In [22]:
!tar -xzvf /DATA/ichuviliaeva/project_data/gazeta_mbart_checkpoint_600_160.tar.gz -C /DATA/ichuviliaeva/project_data/gazeta_bart

checkpoint12.pt


In [33]:
!wget https://www.dropbox.com/s/rqnwjuvp91vhni5/gazeta_data_mbart_600_160.tar.gz \
    -O /DATA/ichuviliaeva/project_data/gazeta_data_mbart_600_160.tar.gz
!wget https://dl.fbaipublicfiles.com/fairseq/models/mbart/mbart.CC25.tar.gz \
    -O /DATA/ichuviliaeva/project_data/mbart.CC25.tar.gz

--2023-04-17 07:38:28--  https://www.dropbox.com/s/rqnwjuvp91vhni5/gazeta_data_mbart_600_160.tar.gz
Resolving www.dropbox.com (www.dropbox.com)... 162.125.70.18, 2620:100:6026:18::a27d:4612
Connecting to www.dropbox.com (www.dropbox.com)|162.125.70.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/rqnwjuvp91vhni5/gazeta_data_mbart_600_160.tar.gz [following]
--2023-04-17 07:38:29--  https://www.dropbox.com/s/raw/rqnwjuvp91vhni5/gazeta_data_mbart_600_160.tar.gz
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc95ee0a2f5befbeea3111755c6d.dl.dropboxusercontent.com/cd/0/inline/B6UOP7ZEF__jAVOsLo31b2nfiYLIH_AWRxvxMxmnYOFjXCvx3nsegMJPtXOXaOlDKNGz0rtaanusz4z2iSOqlygm2yHFmrVEKpu85HRGUodduFfC6C3NjJKWaBgfTdS1Kek11Z7k3SaBvUy0B1g6Qcfs1LHMFuGTADevyF94AkNlYw/file# [following]
--2023-04-17 07:38:29--  https://uc95ee0a2f5befbeea3111755c6d.dl.dropboxusercontent.com/cd/0/inline/B6UOP7ZEF__jAVOsL

In [34]:
!tar -xzvf /DATA/ichuviliaeva/project_data/gazeta_data_mbart_600_160.tar.gz -C /DATA/ichuviliaeva/project_data/gazeta_bart
!tar -xzvf /DATA/ichuviliaeva/project_data/mbart.CC25.tar.gz -C /DATA/ichuviliaeva/project_data/gazeta_bart

data-bin/
data-bin/dict.source.txt
data-bin/dict.target.txt
data-bin/preprocess.log
data-bin/test.source-target.source.bin
data-bin/test.source-target.source.idx
data-bin/test.source-target.target.bin
data-bin/test.source-target.target.idx
data-bin/train.source-target.source.bin
data-bin/valid.source-target.source.bin
data-bin/valid.source-target.source.idx
data-bin/valid.source-target.target.bin
data-bin/valid.source-target.target.idx
data-bin/train.source-target.source.idx
data-bin/train.source-target.target.bin
data-bin/train.source-target.target.idx
mbart.cc25/
mbart.cc25/model.pt
mbart.cc25/sentence.bpe.model
mbart.cc25/dict.txt


In [113]:
!cat fairseq/binarise.sh

fairseq-preprocess \
  --source-lang "source" \
  --target-lang "target" \
  --trainpref "/DATA/ichuviliaeva/project_data/train.bpe" \
  --validpref "/DATA/ichuviliaeva/project_data/val.bpe" \
  --destdir "/DATA/ichuviliaeva/project_data/bin/" \
  --workers 60 \
  --srcdict "/DATA/ichuviliaeva/project_data/gazeta_bart/data-bin/dict.source.txt" \
  --tgtdict "/DATA/ichuviliaeva/project_data/gazeta_bart/data-bin/dict.target.txt";

In [114]:
!bash fairseq/binarise.sh

2023-04-17 10:40:23 | INFO | fairseq_cli.preprocess | Namespace(no_progress_bar=False, log_interval=100, log_format=None, log_file=None, aim_repo=None, aim_run_hash=None, tensorboard_logdir=None, wandb_project=None, azureml_logging=False, seed=1, cpu=False, tpu=False, bf16=False, memory_efficient_bf16=False, fp16=False, memory_efficient_fp16=False, fp16_no_flatten_grads=False, fp16_init_scale=128, fp16_scale_window=None, fp16_scale_tolerance=0.0, on_cpu_convert_precision=False, min_loss_scale=0.0001, threshold_loss_scale=None, amp=False, amp_batch_retries=2, amp_init_scale=128, amp_scale_window=None, user_dir=None, empty_cache_freq=0, all_gather_list_size=16384, model_parallel_size=1, quantization_config_path=None, profile=False, reset_logging=False, suppress_crashes=False, use_plasma_view=False, plasma_path='/tmp/plasma', criterion='cross_entropy', tokenizer=None, bpe=None, optimizer=None, lr_scheduler='fixed', scoring='bleu', task='translation', source_lang='source', target_lang='tar

In [115]:
!cat summarus/external/bart_scripts/preprocess.sh

BART_PATH="$1"

rm -rf "/DATA/ichuviliaeva/project_data/bin/"
fairseq-preprocess \
    --source-lang "source" \
    --target-lang "target" \
    --trainpref "/DATA/ichuviliaeva/project_data/train.bpe" \
    --validpref "/DATA/ichuviliaeva/project_data/val.bpe" \
    --testpref "/DATA/ichuviliaeva/project_data/test.bpe" \
    --destdir "/DATA/ichuviliaeva/project_data/bin/" \
    --workers 60 \
    --srcdict "${BART_PATH}/dict.source.txt" \
    --tgtdict "${BART_PATH}/dict.target.txt";


In [116]:
!bash summarus/external/bart_scripts/preprocess.sh \
/DATA/ichuviliaeva/project_data/gazeta_bart/data-bin

2023-04-17 10:45:49 | INFO | fairseq_cli.preprocess | Namespace(no_progress_bar=False, log_interval=100, log_format=None, log_file=None, aim_repo=None, aim_run_hash=None, tensorboard_logdir=None, wandb_project=None, azureml_logging=False, seed=1, cpu=False, tpu=False, bf16=False, memory_efficient_bf16=False, fp16=False, memory_efficient_fp16=False, fp16_no_flatten_grads=False, fp16_init_scale=128, fp16_scale_window=None, fp16_scale_tolerance=0.0, on_cpu_convert_precision=False, min_loss_scale=0.0001, threshold_loss_scale=None, amp=False, amp_batch_retries=2, amp_init_scale=128, amp_scale_window=None, user_dir=None, empty_cache_freq=0, all_gather_list_size=16384, model_parallel_size=1, quantization_config_path=None, profile=False, reset_logging=False, suppress_crashes=False, use_plasma_view=False, plasma_path='/tmp/plasma', criterion='cross_entropy', tokenizer=None, bpe=None, optimizer=None, lr_scheduler='fixed', scoring='bleu', task='translation', source_lang='source', target_lang='tar

In [117]:
!cat summarus/external/bart_scripts/train.sh

MBART_PATH="$1"
DATA_BIN_PATH="$2"

MAX_UPDATE=80000
WARMUP_UPDATES=500
LR=3e-05
MAX_TOKENS=605
UPDATE_FREQ=2
LANGS=ar_AR,cs_CZ,de_DE,en_XX,es_XX,et_EE,fi_FI,fr_XX,gu_IN,hi_IN,it_IT,ja_XX,kk_KZ,ko_KR,lt_LT,lv_LV,my_MM,ne_NP,nl_XX,ro_RO,ru_RU,si_LK,tr_TR,vi_VN,zh_CN

CUDA_VISIBLE_DEVICES=2 fairseq-train "$DATA_BIN_PATH" \
  --encoder-normalize-before --decoder-normalize-before \
  --arch bart_large --layernorm-embedding \
  --task translation_from_pretrained_bart \
  --source-lang source --target-lang target \
  --criterion label_smoothed_cross_entropy --label-smoothing 0.1 \
  --optimizer adam --adam-eps 1e-06 --adam-betas '(0.9, 0.98)' \
  --lr-scheduler polynomial_decay --lr "$LR" --warmup-updates "$WARMUP_UPDATES" \
  --dropout 0.1 --attention-dropout 0.1 --weight-decay 0.01 --clip-norm 0.1 \
  --total-num-update "$MAX_UPDATE" \
  --max-tokens "$MAX_TOKENS" \
  --required-batch-size-multiple 1 \
  --update-freq "$UPDATE_FREQ" \
  --save-interval 1 --save-interval-updates 5000 \
  --

In [ ]:
!bash summarus/external/bart_scripts/train.sh \
/DATA/ichuviliaeva/project_data/gazeta_bart/checkpoint12.pt \
/DATA/ichuviliaeva/project_data/bin

2023-04-17 10:51:39 | INFO | fairseq_cli.train | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': 'simple', 'log_file': None, 'aim_repo': None, 'aim_run_hash': None, 'tensorboard_logdir': None, 'wandb_project': None, 'azureml_logging': False, 'seed': 42, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': True, 'memory_efficient_fp16': True, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'amp': False, 'amp_batch_retries': 2, 'amp_init_scale': 128, 'amp_scale_window': None, 'user_dir': None, 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parallel_size': 1, 'quantization_config_path': None, 'profile': False, 'reset_logging': False, 'suppress_crashes': False, 'use_plasma_view': False, 'plasma_path': '/tmp/plasma'}, 'common_eval': {'_name'

2023-04-17 10:51:40 | INFO | fairseq.tasks.translation | [source] dictionary: 250001 types
2023-04-17 10:51:40 | INFO | fairseq.tasks.translation | [target] dictionary: 250001 types
2023-04-17 10:51:58 | INFO | fairseq_cli.train | BARTModel(
  (encoder): TransformerEncoderBase(
    (dropout_module): FairseqDropout()
    (embed_tokens): Embedding(250027, 1024, padding_idx=1)
    (embed_positions): LearnedPositionalEmbedding(1026, 1024, padding_idx=1)
    (layernorm_embedding): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
    (layers): ModuleList(
      (0-11): 12 x TransformerEncoderLayerBase(
        (self_attn): MultiheadAttention(
          (dropout_module): FairseqDropout()
          (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
  

2023-04-17 10:52:15 | INFO | fairseq.trainer | NOTE: gradient overflow detected, ignoring gradient, setting loss scale to: 2.0
2023-04-17 10:52:15 | INFO | fairseq.trainer | NOTE: gradient overflow detected, ignoring gradient, setting loss scale to: 1.0
2023-04-17 10:52:16 | INFO | fairseq.trainer | NOTE: gradient overflow detected, ignoring gradient, setting loss scale to: 0.5
/home/ichuviliaeva/miniconda3/envs/is_project/lib/python3.9/site-packages/torch/nn/functional.py:4999: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(
2023-04-17 10:52:23 | INFO | fairseq.trainer | NOTE: gradient overflow detected, ignoring gradient, setting loss scale to: 0.25
2023-04-17 10:52:25 | INFO | fairseq_cli.train | begin validation on "valid" subset
2023-04-17 10:52:25 | INFO | fairseq.tasks.fairseq_task | can_reuse_epoch_itr = True
2023-04-17 10:52:25 | INFO | valid | epoch 001 | valid on 'valid' subset | loss 17.884 |

2023-04-17 10:55:12 | INFO | train | epoch 005 | loss 5.335 | nll_loss 3.223 | ppl 9.34 | wps 613.7 | ups 0.9 | wpb 678.5 | bsz 2 | num_updates 151 | lr 9.06e-06 | gnorm 37.139 | clip 100 | loss_scale 0.25 | train_wall 12 | gb_free 13.9 | wall 192
2023-04-17 10:55:12 | INFO | fairseq.tasks.fairseq_task | can_reuse_epoch_itr = True
2023-04-17 10:55:12 | INFO | fairseq.data.iterators | grouped total_num_itrs = 32
2023-04-17 10:55:12 | INFO | fairseq.trainer | begin training epoch 6
2023-04-17 10:55:12 | INFO | fairseq_cli.train | Start iterating over samples
2023-04-17 10:55:25 | INFO | fairseq_cli.train | begin validation on "valid" subset
2023-04-17 10:55:25 | INFO | fairseq.tasks.fairseq_task | can_reuse_epoch_itr = True
2023-04-17 10:55:25 | INFO | valid | epoch 006 | valid on 'valid' subset | loss 4.938 | nll_loss 3.109 | ppl 8.63 | wps 6992 | wpb 290.8 | bsz 1 | num_updates 183 | best_loss 4.938
2023-04-17 10:55:25 | INFO | fairseq.checkpoint_utils | Preparing to save checkpoint fo

2023-04-17 10:58:08 | INFO | fairseq_cli.train | begin validation on "valid" subset
2023-04-17 10:58:08 | INFO | fairseq.tasks.fairseq_task | can_reuse_epoch_itr = True
2023-04-17 10:58:09 | INFO | valid | epoch 011 | valid on 'valid' subset | loss 4.556 | nll_loss 2.635 | ppl 6.21 | wps 6135.9 | wpb 290.8 | bsz 1 | num_updates 343 | best_loss 4.556
2023-04-17 10:58:09 | INFO | fairseq.checkpoint_utils | Preparing to save checkpoint for epoch 11 @ 343 updates
2023-04-17 10:58:09 | INFO | fairseq.trainer | Saving checkpoint to /home/ichuviliaeva/is_project/checkpoints/checkpoint_best.pt
2023-04-17 10:58:22 | INFO | fairseq.trainer | Finished saving checkpoint to /home/ichuviliaeva/is_project/checkpoints/checkpoint_best.pt
2023-04-17 10:58:30 | INFO | fairseq.checkpoint_utils | Saved checkpoint checkpoints/checkpoint_best.pt (epoch 11 @ 343 updates, score 4.556) (writing took 21.62903234793339 seconds)
2023-04-17 10:58:30 | INFO | fairseq_cli.train | end of epoch 11 (average epoch stats 

...

2023-04-17 16:36:40 | INFO | fairseq.checkpoint_utils | Saved checkpoint checkpoints/checkpoint_last.pt (epoch 815 @ 26071 updates, score 6.179) (writing took 12.269256567000411 seconds)
2023-04-17 16:36:40 | INFO | fairseq_cli.train | end of epoch 815 (average epoch stats below)
2023-04-17 16:36:40 | INFO | train | epoch 815 | loss 2.339 | nll_loss 0.189 | ppl 1.14 | wps 918.8 | ups 1.35 | wpb 678.5 | bsz 2 | num_updates 26071 | lr 2.03506e-05 | gnorm 1.897 | clip 100 | loss_scale 2 | train_wall 11 | gb_free 13.9 | wall 20680
2023-04-17 16:36:40 | INFO | fairseq.tasks.fairseq_task | can_reuse_epoch_itr = True
2023-04-17 16:36:40 | INFO | fairseq.data.iterators | grouped total_num_itrs = 32
2023-04-17 16:36:40 | INFO | fairseq.trainer | begin training epoch 816
2023-04-17 16:36:40 | INFO | fairseq_cli.train | Start iterating over samples
2023-04-17 16:36:50 | INFO | train_inner | epoch 816:     29 / 32 loss=2.339, nll_loss=0.191, ppl=1.14, wps=947.1, ups=1.39, wpb=683.1, bsz=2, num_upd

2023-04-17 16:38:40 | INFO | fairseq.trainer | Finished saving checkpoint to /home/ichuviliaeva/is_project/checkpoints/checkpoint_last.pt
2023-04-17 16:38:40 | INFO | fairseq.checkpoint_utils | Saved checkpoint checkpoints/checkpoint_last.pt (epoch 820 @ 26231 updates, score 6.121) (writing took 12.888088098028675 seconds)
2023-04-17 16:38:40 | INFO | fairseq_cli.train | end of epoch 820 (average epoch stats below)
2023-04-17 16:38:40 | INFO | train | epoch 820 | loss 2.338 | nll_loss 0.191 | ppl 1.14 | wps 869 | ups 1.28 | wpb 678.5 | bsz 2 | num_updates 26231 | lr 2.02902e-05 | gnorm 2.02 | clip 100 | loss_scale 2 | train_wall 12 | gb_free 13.9 | wall 20800
2023-04-17 16:38:40 | INFO | fairseq.tasks.fairseq_task | can_reuse_epoch_itr = True
2023-04-17 16:38:40 | INFO | fairseq.data.iterators | grouped total_num_itrs = 32
2023-04-17 16:38:40 | INFO | fairseq.trainer | begin training epoch 821
2023-04-17 16:38:40 | INFO | fairseq_cli.train | Start iterating over samples
2023-04-17 16:3

2023-04-17 16:40:39 | INFO | fairseq.checkpoint_utils | Saved checkpoint checkpoints/checkpoint_last.pt (epoch 825 @ 26391 updates, score 6.148) (writing took 11.96198808902409 seconds)
2023-04-17 16:40:39 | INFO | fairseq_cli.train | end of epoch 825 (average epoch stats below)
2023-04-17 16:40:39 | INFO | train | epoch 825 | loss 2.339 | nll_loss 0.192 | ppl 1.14 | wps 925.2 | ups 1.36 | wpb 678.5 | bsz 2 | num_updates 26391 | lr 2.02298e-05 | gnorm 2.109 | clip 100 | loss_scale 2 | train_wall 11 | gb_free 13.9 | wall 20919
2023-04-17 16:40:39 | INFO | fairseq.tasks.fairseq_task | can_reuse_epoch_itr = True
2023-04-17 16:40:39 | INFO | fairseq.data.iterators | grouped total_num_itrs = 32
2023-04-17 16:40:39 | INFO | fairseq.trainer | begin training epoch 826
2023-04-17 16:40:39 | INFO | fairseq_cli.train | Start iterating over samples
2023-04-17 16:40:42 | INFO | train_inner | epoch 826:      9 / 32 loss=2.339, nll_loss=0.192, ppl=1.14, wps=933.1, ups=1.39, wpb=670, bsz=2, num_update

2023-04-17 16:42:36 | INFO | fairseq.trainer | Finished saving checkpoint to /home/ichuviliaeva/is_project/checkpoints/checkpoint_last.pt
2023-04-17 16:42:37 | INFO | fairseq.checkpoint_utils | Saved checkpoint checkpoints/checkpoint_last.pt (epoch 830 @ 26551 updates, score 6.219) (writing took 12.14427742396947 seconds)
2023-04-17 16:42:37 | INFO | fairseq_cli.train | end of epoch 830 (average epoch stats below)
2023-04-17 16:42:37 | INFO | train | epoch 830 | loss 2.338 | nll_loss 0.19 | ppl 1.14 | wps 924.4 | ups 1.36 | wpb 678.5 | bsz 2 | num_updates 26551 | lr 2.01694e-05 | gnorm 1.864 | clip 100 | loss_scale 2 | train_wall 11 | gb_free 13.9 | wall 21037
2023-04-17 16:42:37 | INFO | fairseq.tasks.fairseq_task | can_reuse_epoch_itr = True
2023-04-17 16:42:37 | INFO | fairseq.data.iterators | grouped total_num_itrs = 32
2023-04-17 16:42:37 | INFO | fairseq.trainer | begin training epoch 831
2023-04-17 16:42:37 | INFO | fairseq_cli.train | Start iterating over samples
2023-04-17 16:

2023-04-17 16:44:34 | INFO | fairseq.trainer | Finished saving checkpoint to /home/ichuviliaeva/is_project/checkpoints/checkpoint_last.pt
2023-04-17 16:44:34 | INFO | fairseq.checkpoint_utils | Saved checkpoint checkpoints/checkpoint_last.pt (epoch 835 @ 26711 updates, score 6.135) (writing took 12.17313062993344 seconds)
2023-04-17 16:44:34 | INFO | fairseq_cli.train | end of epoch 835 (average epoch stats below)
2023-04-17 16:44:34 | INFO | train | epoch 835 | loss 2.338 | nll_loss 0.19 | ppl 1.14 | wps 918.3 | ups 1.35 | wpb 678.5 | bsz 2 | num_updates 26711 | lr 2.01091e-05 | gnorm 2.125 | clip 100 | loss_scale 2 | train_wall 11 | gb_free 13.9 | wall 21155
2023-04-17 16:44:34 | INFO | fairseq.tasks.fairseq_task | can_reuse_epoch_itr = True
2023-04-17 16:44:34 | INFO | fairseq.data.iterators | grouped total_num_itrs = 32
2023-04-17 16:44:34 | INFO | fairseq.trainer | begin training epoch 836
2023-04-17 16:44:34 | INFO | fairseq_cli.train | Start iterating over samples
2023-04-17 16:

In [1]:
%ls

bart_preprocess.sh  ru_articles_summarization/             summarus/
fairseq/            ru-articles-summarization-mbart.ipynb


In [2]:
%cd summarus

/home/ichuviliaeva/is_project/summarus


In [2]:
import os

In [3]:
os.environ['CUDA_VISIBLE_DEVICES'] = '2'

In [5]:
import json
import torch
from tqdm.notebook import tqdm
from transformers import MBartTokenizer, MBartForConditionalGeneration


def gen_batch(inputs, batch_size):
    batch_start = 0
    while batch_start < len(inputs):
        yield inputs[batch_start: batch_start + batch_size]
        batch_start += batch_size


def predict(
    model_name,
    test_file,
    predictions_file,
    targets_file,
    max_source_tokens_count=600,
    max_target_tokens_count=160,
    use_cuda=True,
    batch_size=4
):
    inputs = []
    targets = []
    
    with open(test_file, "r") as r:
        for line in r:
            record = json.loads(line)
            inputs.append(record["article"])
            targets.append(record["abstract"].replace("\n", " "))
        
    tokenizer = MBartTokenizer.from_pretrained(model_name)
    device = torch.device("cuda:0") if use_cuda else torch.device("cpu")
    model = MBartForConditionalGeneration.from_pretrained(model_name).to(device)
    # print('model = ', model)
    # print('test_file = ', test_file)
    # print('predictions_file = ', predictions_file)
    # print('targets_file = ', targets_file)
    
    predictions = []
    batches = gen_batch(inputs, batch_size)
    for batch in tqdm(batches):
        input_ids = tokenizer.prepare_seq2seq_batch(
            batch,
            src_lang="ru_RU",
            return_tensors="pt",
            padding="max_length",
            truncation=True,
            max_length=max_source_tokens_count
        )["input_ids"].to(device)
        output_ids = model.generate(
            input_ids=input_ids,
            max_length=max_target_tokens_count + 2,
            no_repeat_ngram_size=3,
            num_beams=5,
            top_k=0
        )
        summaries = tokenizer.batch_decode(output_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)
        predictions.extend(summaries)
        
    with open(predictions_file, "w") as w:
        for p in predictions:
            w.write(p.strip() + "\n")
    with open(targets_file, "w") as w:
        for t in targets:
            w.write(t.strip() + "\n")
        
    return predictions, targets

In [6]:
predictions, targets = predict(
    "IlyaGusev/mbart_ru_sum_gazeta", 
    "/DATA/ichuviliaeva/project_data/test_data.json",
    "/DATA/ichuviliaeva/project_data/predictions.txt", 
    "/DATA/ichuviliaeva/project_data/targets.txt", 
    batch_size = 16
)

0it [00:00, ?it/s]

/home/ichuviliaeva/miniconda3/envs/is_project/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:3712: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and targets.

Here is a short example:

model_inputs = tokenizer(src_texts, text_target=tgt_texts, ...)

If you either need to use different keyword arguments for the source and target texts, you should do two calls like
this:

model_inputs = tokenizer(src_texts, ...)
labels = tokenizer(text_target=tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)


In [ ]:
# работает около часа

In [13]:
%cd is_project

/home/ichuviliaeva/is_project


In [4]:
%ls

bart_preprocess.sh  ru_articles_summarization/             summarus/
fairseq/            ru-articles-summarization-mbart.ipynb


In [5]:
!cd fairseq

In [16]:
import json
import torch
from tqdm.notebook import tqdm
from transformers import MBartTokenizer, MBartForConditionalGeneration

def train(
    model_name,
    train_file,
    val_file,
    max_source_tokens_count=600,
    max_target_tokens_count=160,
    use_cuda=True,
    batch_size=4
):
    inputs_train = []
    targets_train = []
    inputs_val = []
    targets_val = []

    with open(train_file, "r") as r:
        for line in tqdm(r):
            record = json.loads(line)
            inputs_train.append(record["article"])
            targets_train.append(record["abstract"].replace("\n", " "))
    
    with open(val_file, "r") as r:
        for line in tqdm(r):
            record = json.loads(line)
            inputs_val.append(record["article"])
            targets_val.append(record["abstract"].replace("\n", " "))
    
    tokenizer = MBartTokenizer.from_pretrained(model_name)
    device = torch.device("cuda:0") if use_cuda else torch.device("cpu")
    model = MBartForConditionalGeneration.from_pretrained(model_name).to(device)
    model.train()
    
    predictions = []
    train_batches = gen_batch(inputs_train, batch_size)
    
    for batch in tqdm(train_batches):
        input_ids = tokenizer.prepare_seq2seq_batch(
            batch,
            src_lang="ru_RU",
            return_tensors="pt",
            padding="max_length",
            truncation=True,
            max_length=max_source_tokens_count
        )["input_ids"].to(device)
        output_ids = model.generate(
            input_ids=input_ids,
            max_length=max_target_tokens_count + 2,
            no_repeat_ngram_size=3,
            num_beams=5,
            top_k=0
        )
        summaries = tokenizer.batch_decode(output_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)
        predictions.extend(summaries)
        
    # можно попытаться вставить лосс и оптимизатор вот сюда
    
    val_batches = gen_batch(inputs_val, batch_size)
    
    for batch in tqdm(val_batches):
        input_ids = tokenizer.prepare_seq2seq_batch(
            batch,
            src_lang="ru_RU",
            return_tensors="pt",
            padding="max_length",
            truncation=True,
            max_length=max_source_tokens_count
        )["input_ids"].to(device)
        output_ids = model.generate(
            input_ids=input_ids,
            max_length=max_target_tokens_count + 2,
            no_repeat_ngram_size=3,
            num_beams=5,
            top_k=0
        )
        summaries = tokenizer.batch_decode(output_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)
        predictions.extend(summaries)
        
    return predictions, targets


In [17]:
predictions, targets = train("IlyaGusev/mbart_ru_sum_gazeta", 
                             "/DATA/ichuviliaeva/project_data/train_data.json",
                             "/DATA/ichuviliaeva/project_data/val_data.json"
                            )

0it [00:00, ?it/s]

KeyboardInterrupt: 